# Salary Predictions Based on Job Descriptions

# Step 4: Deploy

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

In [2]:
import sys
sys.path.insert(0, './scripts')

from helpers import load_data
from Preprocessing import Preprocessing

## Automating the Pipeline

In [3]:
path = './cleaned_data/'
files = ['cleaned.csv']
train_data = pd.DataFrame()

for file in files:
    train_data = data.append(load_data(path, file))

print(train_data.shape)
train_data.head()

(991513, 8)


,salary,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
jobId,,,,,,,,
JOB1362684407687,130,COMP37,CFO,MASTERS,MATH,HEALTH,10,83
JOB1362684407688,101,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73
JOB1362684407689,137,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38
JOB1362684407690,142,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17
JOB1362684407691,163,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16


In [7]:
path = './raw_data/'
file = 'test_features.csv'
features_test = load_data(path, file)

In [6]:
target = ['salary']
features = [col for col in data.columns if col not in target]

features_train = data[features]
salary_train = data[target]

In [5]:
cols_to_filter = ['companyId']
ordinal_cols = {'jobType': ['JANITOR', 'JUNIOR', 'SENIOR', 'MANAGER', 'VICE_PRESIDENT', 'CFO', 'CTO', 'CEO'],
                'degree': ['NONE', 'HIGH_SCHOOL', 'BACHELORS', 'MASTERS', 'DOCTORAL']
               }
binned_cols = {'yearsExperience': 5}
p = Preprocessing(cols_to_filter=cols_to_filter, ordinal_cols=ordinal_cols,
                  binned_cols=binned_cols, grad_types=True,
                  combo=True                          
                 )
rf = RandomForestRegressor(max_depth=25, min_samples_leaf=17, max_features=.35,
                           n_estimators=100, n_jobs=-1, random_state=44        
                          )

pipeline = Pipeline([('preprocessing', p), ('Random Forest', rf)])

## Deploying the solution

In [ ]:
pipeline.fit(features_train, salary_train.values.ravel())
salary_pred = pipeline.predict(features_test)

## Measuring Efficacy 
I cannot perform this last step because I do not have the solutions to the testing data.